In [61]:
from polychrom.hdf5_format import HDF5Reporter
from polychrom.polymerutils import fetch_block
from polychrom import forcekits, forces, starting_conformations, simulation
from polychrom.hdf5_format import HDF5Reporter
import numpy as np
import openmm as mm
from dataclasses import dataclass, field
from math import pi
import pickle 
import math
from typing import Dict, List, Tuple, TypedDict, Any, Sequence

import numpy as np


from poly_shared import (
    MoleculeStructure,
    Bond,
    Angle,
    Dihedral,
    dihedral_angle,
    verify_bond_angle_dihedral_uniqeness,
    concatenate_molecule_structures,
    build_ribbon,
    connect_ribbon_ends,
    connect_with_polymer,
    add_polymer_forces,
)


In [69]:
r = 15.0
# monomer count around ring ≈ circumference/step
n_ring = int(2*math.pi*r/0.9)
ring = build_hairy_ring(r, n_ring, p_hairy=0.1, seed=42, start_particle_index=0)
ring_s = ring.start_particle_index
ring_e = ring.get_end_idx()


# attach two lines length=20 at roughly opposite points
half = n_ring//2
idxA = ring.index_dict[(0,0)]
idxB = ring.index_dict[(half,0)]


l1s, l1e = attach_line(ring, attach_idx=idxA, n_monomers=25, seed=1)
l2s, l2e = attach_line(ring, attach_idx=idxB, n_monomers=25, seed=2)


In [70]:
from openmm import CustomManyParticleForce

force_eq = """
r = (
r12 = distance(p1, p2); r13 = distance(p1, p3); r23 = distance(p2, p3)
"""


# CustomManyParticleForce* force = new CustomManyParticleForce(3,
#      "L*eps*(cos(theta1)+1/3)^2*exp(sigma*gamma/(r12-a*sigma))*exp(sigma*gamma/(r13-a*sigma));"
#        "r12 = distance(p1,p2); r13 = distance(p1,p3); theta1 = angle(p3,p1,p2)");
#  force->setPermutationMode(CustomManyParticleForce::UniqueCentralParticle);

In [116]:
# combined_structure = concatenate_molecule_structures(
#     [ring],
#     extra_bonds=[],
#     extra_angles=[],
#     extra_dihedrals=[],
# )


# manually construct molecular structure for a length=10 line in the X,Y,Z direction from (1,0,0), etc.
index = 0 
lines = []
for dim in range(3):

    pos = np.zeros((40, 3), float)
    bonds = []
    angles = []
    for i in range(40):
        pos[i, dim] = 0.7 + i * 0.85  # X direction, 0.9 is the step size
        if i > 0:
            bonds.append((i+index - 1, i+index, 1, 0.05))
        if i > 1:
            angles.append((i+index - 2, i+index - 1, i+index, math.pi, 2.))
    dummmyIndexDict = {(0,i):i + index for i in range(40)}
    line1 = MoleculeStructure(
        n_rows=1,
        start_particle_index=index,
        positions=pos,
        bonds=bonds,
        angles=angles,
        dihedrals=[],
        index_dict=dummmyIndexDict,
    )
    index += 40
    lines.append(line1)

combined_structure = concatenate_molecule_structures(
    lines)





N = len(combined_structure.positions)
positions = combined_structure.positions.copy()
bonds = combined_structure.bonds.copy()
angles = combined_structure.angles.copy()
dihedrals = combined_structure.dihedrals.copy()


positions = pickle.load(open("spider.pkl", "rb"))

reporter = HDF5Reporter(folder="trajectory", max_data_length=5, overwrite=True)
sim = simulation.Simulation(
    platform="OpenCL",
    integrator="variableLangevin",
    error_tol=0.001,
    GPU="0",
    collision_rate=0.02,
    N=N,
    save_decimals=2,
    PBCbox=False,
    reporters=[reporter],
)


sim.set_data(positions, center=True)  # loads a polymer, puts a center of mass at zero
# Add forces to the simulation
res = add_polymer_forces(sim, bonds, angles, dihedrals)

# sim.add_force(forces.spherical_confinement(sim, r=5, k=1))
# nonbonded force
rep_force = forces.polynomial_repulsive(sim, trunc=50)
sim.add_force(rep_force)

type1_indices = list(range(40)) # entire first chain vs start of chain 2,3
type2_indices = [40,41]
type3_indices = [80,81]

three_way_force = threeWayAttraction(
    sim,
    type1_particle_idx=type1_indices,
    type2_particle_idx=type2_indices,
    type3_particle_idx=type3_indices,
    attractionEnergy=2.2,
    attractionRadius=2.0,
)
sim.add_force(three_way_force)


remover = mm.CMMotionRemover()
remover.name = "remover"  # type: ignore - this is polychrom's old convention
sim.add_force(remover)  # add motion remover to remove center of mass motion

sim.eK_critical = 1000

sim.local_energy_minimization()
#sim.local_energy_minimization()
#sim.local_energy_minimization()

all_dh = []
for block in range(100):
    sim.do_block(
        1 if block == 0 else 2000, save_extras={"bonds": bonds}
    )  # Of 100 timesteps each. Data is saved automatically.
    pos = sim.get_data()


sim.print_stats()  # In the end, print very simple statistics

reporter.dump_data()  # always need to run in the end to dump the block cache to the disk

INFO:root:Performing local energy minimization
INFO:root:adding force harmonic_bonds 0
INFO:root:adding force angle 1
INFO:root:adding force polynomial_repulsive 2
INFO:root:adding force three_way_attraction 3
INFO:root:adding force remover 4
INFO:root:Particles loaded. Potential energy is 126.191819
INFO:root:before minimization eK=1.572402133345948, eP=126.19181256189901, time=0.0 ps
INFO:root:Particles loaded. Potential energy is 0.283120
INFO:root:after minimization eK=1.572402133345948, eP=-0.509070549612843, time=0.0 ps
INFO:root:block    0 pos[1]=[-4.3 0.4 -2.7] dr=0.01 t=0.0ps kin=1.85 pot=0.05 Rg=11.755 SPS=353 dt=21.8fs dx=6.60pm 
INFO:root:block    1 pos[1]=[-3.9 -0.1 -1.4] dr=4.47 t=58.3ps kin=1.65 pot=1.45 Rg=11.479 SPS=1835 dt=30.0fs dx=8.61pm 
INFO:root:block    2 pos[1]=[-4.2 -0.4 0.0] dr=3.29 t=113.0ps kin=1.96 pot=1.41 Rg=11.058 SPS=2197 dt=26.3fs dx=8.23pm 
INFO:root:block    3 pos[1]=[-4.3 -1.5 -0.5] dr=3.82 t=167.3ps kin=2.11 pot=1.55 Rg=10.292 SPS=2717 dt=27.8fs d


 Statistics: number of particles: 120

Statistics for particle position
     mean position is:  [ 0.00299293 -0.00573332  0.0013068 ]   Rg =  18.684143
     median bond size is  1.0212589221999708
     three shortest/longest (<10)/ bonds are  [0.977577   0.97775358 0.97882581]    [1.11039952 1.11238965 1.14425208]
longest 10 bonds are [ 1.08108007  1.08667219  1.08715554  1.09565889  1.09694471  1.11039952
  1.11238965  1.14425208 24.44112877 30.15936364]
     95 percentile of distance to center is:    22.5715449015009
     density of closest 95% monomers is:    0.002366641012681677
     density of the 5% closest to CoM monomers is:    0.0007870458709053448
     min/median/mean/max coordinates are: 
     x: -18.79, -2.41, 0.00, 22.51
     y: -12.49, -4.73, -0.01, 17.20
     z: -12.67, -2.92, 0.00, 15.29

Statistics for velocities:
     mean kinetic energy is:  2.1809106408964465 should be: 1.5
     fastest particles are (in kT):  [ 5.4890064   6.59648949  7.8455139   9.39505211 10.993

In [ ]:
twf = three_way_force
twf.getTypeFilter(2)

(4,)

In [ ]:
# data = sim.get_data() 
# pickle.dump(data, open("scrunched.pkl", "wb"))  # save the data to a file

In [ ]:
dh = np.array(all_dh)
np.mean(dh, axis=0)  # mean dihedral angles

/opt/homebrew/Caskroom/miniconda/base/envs/py313/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniconda/base/envs/py313/lib/python3.13/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


np.float64(nan)

In [ ]:

import matplotlib.pyplot as plt
plt.figure() 
plt.scatter(dh[:, 0], dh[:, 2], s=1,  cmap="viridis")

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

<Figure size 640x480 with 0 Axes>